# Fleet Management - Live Demo

In this notebook we'll show how you can follow an object with JetBot!  We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices).  The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
which provides utilities for training object detectors for custom tasks also!  Once the model is trained, we optimize it using NVIDIA TensorRT on the Jetson Nano.

This makes the network very fast, capable of real-time execution on Jetson Nano!  We won't run through all of the training and optimization steps in this notebook though.

Anyways, let's get started.  First, we'll want to import the ``Object_Follower`` class which takes our pre-trained SSD engine or yolo engine.

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
from ipywidgets.widgets import Box, HBox, VBox, Layout, Label
from IPython.display import clear_output
import traitlets


In [ ]:
# from main_scripts import Object_Follower
from jetbot import Fleeter

type_follower_model = "YOLO"  # "SSD", "YOLO"
# follower_model='ssd_mobilenet_v2_coco_onnx.engine'
# follower_model='yolov3_tiny_288.engine'
# follower_model='yolov4_tiny_288.engine'
# follower_model='yolov4_tiny_416.eigine'
# follower_model='yolov4_288.engine'
follower_model='yolov4_416.engine'

type_cruiser_model = "resnet"
cruiser_model='resnet18'

FL = Fleeter(follower_model=follower_model, type_follower_model=type_follower_model, 
             cruiser_model=cruiser_model, type_cruiser_model=type_cruiser_model)

The followings is to construct the control widgets for fleet control.

In [ ]:
out = widgets.Output()

# image_widget = widgets.Image(format='jpeg', width=OF.img_width, height=OF.img_height)
image_widget = widgets.Image(format='jpeg', width=300, height=300, layout = Layout(align_self='center'))

# display(image_widget)
traitlets.dlink((FL, 'cap_image'), (image_widget, 'value'))

# display buttons
button_layout = widgets.Layout(width='100px', height='40px', align_self='center')
stop_button = widgets.Button(description='Stop', button_style='danger', tooltip='Click to stop running', icon='fa-stop', layout=button_layout)
start_button = widgets.Button(description='Start', tooltip='Click to start running', layout=button_layout)
button_box = widgets.HBox([start_button, stop_button], layout=widgets.Layout(justify_content='space-around', width='30%'))


In [ ]:
# infos and params of fleeting
blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
object_view_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='object view')
label_widget = widgets.IntText(value=1, description='tracked label')  # target to be tracked
label_text_widget = widgets.Text(value='', description='label name')  # target name to be tracked
speed_widget = widgets.FloatSlider(min=0.0, max=1.0, description='speed', readout_format='.3f')
speed_gain_widget = widgets.FloatSlider(value=0.01, min=0.0, max=0.05, step=0.01, description='speed_gain', readout_format='.3f')
speed_dev_widget = widgets.FloatSlider(value=0.5, min=0.05, max=1, step=0.01, description='speed_dev', readout_format='.3f')
turn_gain_widget = widgets.FloatSlider(value=0.25, min=0.05, max=0.5, step=0.001, description='turn gain', readout_format='.3f')
steering_bias_widget = widgets.FloatSlider(value=0.02, min=-0.1, max=0.1, step=0.001, description='steering bias', readout_format='.3f')
view_target_widget = widgets.FloatSlider(value=0.3, min=0.001, max=1.0, step=0.001, description='view target', readout_format='.3f')

traitlets.dlink((FL, 'blocked'), (blocked_widget, 'value'))
traitlets.dlink((FL, 'mean_view'), (object_view_widget, 'value'))
traitlets.dlink((label_widget, 'value'), (FL, 'label'))
traitlets.dlink((FL, 'label_text'), (label_text_widget, 'value'))
traitlets.dlink((turn_gain_widget, 'value'), (FL, 'turn_gain'))
traitlets.dlink((speed_gain_widget, 'value'), (FL, 'speed_gain'))
traitlets.dlink((speed_dev_widget, 'value'), (FL, 'speed_dev'))
traitlets.dlink((FL, 'speed'), (speed_widget, 'value'))
traitlets.dlink((steering_bias_widget, 'value'), (FL, 'steering_bias'))
traitlets.dlink((view_target_widget, 'value'), (FL, 'target_view'))

In [ ]:
# control params of road cruising
speed_gain_slider = widgets.FloatSlider(min=0, max=1, step=0.001, value=0.2, description='speed gain', readout_format='.3f')
steering_gain_slider = widgets.FloatSlider(min=0, max=0.5, step=0.001, value=0.08, description='steering gain', readout_format='.3f')
steering_dgain_slider = widgets.FloatSlider(min=0, max=2.0, step=0.001, value=0.82, description='steering kd', readout_format='.3f')
steering_bias_slider = widgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=-0.01, description='steering bias', readout_format='.3f')

traitlets.dlink((speed_gain_slider, 'value'), (FL.road_cruiser, 'speed_gain'))
traitlets.dlink((steering_gain_slider, 'value'), (FL.road_cruiser, 'steering_gain'))
traitlets.dlink((steering_dgain_slider, 'value'), (FL.road_cruiser, 'steering_dgain'))
traitlets.dlink((steering_bias_slider, 'value'), (FL.road_cruiser, 'steering_bias'))

# VBox_image = VBox([image_widget], layout=Layout(align_self='center'))
VBox_control = VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider], layout=Layout(align_self='center'))

# stat information of road cruising
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = widgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='vertical', description='speed')

traitlets.dlink((FL.road_cruiser, 'x_slider'), (x_slider, 'value'))
traitlets.dlink((FL.road_cruiser, 'y_slider'), (y_slider, 'value'))
traitlets.dlink((FL.road_cruiser, 'steering'), (steering_slider, 'value'))
traitlets.dlink((FL.road_cruiser, 'speed'), (speed_slider, 'value'))

Box_y_state = HBox([y_slider, speed_slider])
Box_x_state = VBox([x_slider, steering_slider])
Box_state = VBox([Box_y_state, Box_x_state])


In [ ]:
def start(change):
    FL.start_run(change)

def stop(change):
    FL.stop_run(change)
    %reset -f

In [ ]:
RC_box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid 2px', width='60%')
FL_box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid 2px', width='60%')
H_box_layout = Layout(display='flex', flex_flow='row', align_items='stretch')
label_layout = Layout(display='flex', flex_flow='row', align_items='stretch', align_self='center')

# fleet control widgets 
Fleet_Control_items = [HBox([Label(value='--- Fleet Control ---')], layout=Layout(align_self='center')),
                       HBox([blocked_widget, object_view_widget], layout = H_box_layout),
                       HBox([speed_widget, view_target_widget], layout = H_box_layout),
                       HBox([speed_gain_widget, speed_dev_widget], layout = H_box_layout),
                       HBox([turn_gain_widget, steering_bias_widget], layout = H_box_layout)]

RC_control_items = [HBox([Label(value='--- Cruising Control ---')], layout=Layout(align_self='center')),
                    HBox([Box_state, VBox_control], layout = H_box_layout)]

image_items = [image_widget, VBox([label_text_widget, label_widget])]

display(HBox([VBox(children = image_items, layout = Layout(align_self='center')),
             Box(children = RC_control_items, layout = RC_box_layout)], layout = Layout(justify_content='space-between')),
        HBox([button_box, Box(children = Fleet_Control_items, layout = FL_box_layout)], layout = Layout(align_items='stretch', justify_content='space-between')))

start_button.on_click(start)
stop_button.on_click(stop)